# Introduction

In this notebook we will use the code for finding the optimal node positions in a simple Instagram network visualization of `visualize_network.ipynb`, but here we will concentrate on making the code portable, by handling cases when there are very few or very many tags:
* If there are no tags, just put the central node's image in the center
* If there is only one tag, put it next to the central node, so they are centered in the image
* If there are more than 2 tags, check if the number of tags is larger than a maximal number of nodes you can theoretically put in the figure, and if so, get rid of the extra tags
* Then, check if there is overlap (one node on top of another), and if so, get rid of the last node (i.e. the one with the smallest amount of tags) and recalculate - continue do so until there is no overlap

It turns out it's faster and more practical to handle resizing and placing images by HTML's `canvas` in the ARRA app, so this code will spit out the optimal node positions as a list containing the pixel values of those positions.

# All the functions

In [8]:
import numpy as np

In [9]:
def generate_relative_graph(nNodes, r_func, size_sec):
    # Based on the given radial distribution of nodes, 
    # generate relative node positions around the central one
    x_pos = []
    y_pos = []
    for n in range(nNodes - 1):
        x_pos = x_pos + [r_func[n] * np.cos(n * 2*np.pi / (nNodes - 1))]
        y_pos = y_pos + [r_func[n] * np.sin(n * 2*np.pi / (nNodes - 1))]
    graph_width = max(x_pos) - min(x_pos) + size_sec
    graph_height = max(y_pos) - min(y_pos) + size_sec
    return (x_pos, y_pos, graph_width, graph_height)

In [10]:
def case1_positions(r_min, tag_list, size_sec, fig_width, fig_height, index_list):
    # Calculate final relative node positions in Case 1
    # (see 'visualize_network.ipynb')
    nNodes = len(tag_list)
    graph_too_big = False
    new_r_min = r_min
    max_tags = max(tag_list[1:])
    while not graph_too_big:
        new_r_min = new_r_min + 1
        r_trial = (new_r_min * max_tags) / np.array(tag_list[1:])
        (x_pos, y_pos, graph_width, graph_height) = generate_relative_graph(nNodes, r_trial, size_sec)
        if (graph_width > fig_width) | (graph_height > fig_height): graph_too_big = True
    new_r_min = new_r_min - 2
    r_trial = (new_r_min * max_tags) / np.array(tag_list[1:])
    (x_pos, y_pos, graph_width, graph_height) = generate_relative_graph(nNodes, r_trial, size_sec)
    return (x_pos, y_pos, graph_width, graph_height)

In [11]:
def case2_positions(r_min, tag_list, size_sec, fig_width, fig_height, index_list):
    # Calculate final relative node positions in Case 2
    # (see 'visualize_network.ipynb')
    nNodes = len(tag_list)
    graph_too_big = False
    r_max = r_min
    min_tags = min(tag_list[1:])
    max_tags = max(tag_list[1:])
    while not graph_too_big:
        r_max = r_max + 1
        rad_slope = (r_min - r_max) / (max_tags - min_tags)
        rad_intercept = (r_max * max_tags - r_min * min_tags) / (max_tags - min_tags)
        this_tags = np.array(tag_list[1:])
        r_trial = rad_slope * this_tags + rad_intercept
        (x_pos, y_pos, graph_width, graph_height) = generate_relative_graph(nNodes, r_trial, size_sec)
        if (graph_width > fig_width) | (graph_height > fig_height): graph_too_big = True
    r_max = r_max - 2
    rad_slope = (r_min - r_max) / (max_tags - min_tags)
    rad_intercept = (r_max * max_tags - r_min * min_tags) / (max_tags - min_tags)
    this_tags = np.array(tag_list[1:])
    r_trial = rad_slope * this_tags + rad_intercept
    (x_pos, y_pos, graph_width, graph_height) = generate_relative_graph(nNodes, r_trial, size_sec)
    return (x_pos, y_pos, graph_width, graph_height)

In [12]:
def node_positions(input_tag_list, fig_width, fig_height, size_prim, size_sec, r_min):
    # Initial ordering of the tag list, saving their indexes
    tag_list = input_tag_list
    index_list = range(len(tag_list))
    zip_list = [(0, 0)] + sorted(zip(tag_list[1:], index_list[1:]), reverse = True)
    index_list = [index for tag, index in zip_list]
    tag_list = [tag for tag, index in zip_list]
    # Cut off all the nodes we know won't able to fit
    n_max = int((fig_width / size_sec) * (fig_height / size_sec)) + 1
    if len(tag_list) > n_max:
        tag_list = tag_list[0:n_max]
        index_list = index_list[0:n_max]
    # Start checking for overlap
    overlap = True
    while overlap == True:
        nNodes = len(tag_list)
        # Check which case we have
        if nNodes == 1:
            node_pos = [0, int(round(fig_width / 2)), int(round(fig_height / 2))]
        if nNodes == 2:
            first_node = [[0, int(round((fig_width - r_min) / 2)), int(round(fig_height / 2))]]
            second_node = [[1, int(round((fig_width + r_min) / 2)), int(round(fig_height / 2))]]
            node_pos = first_node + second_node
        if nNodes > 2:
            min_tags = min(tag_list[1:])
            max_tags = max(tag_list[1:])
            r_trial = (r_min * max_tags) / np.array(tag_list[1:])
            (x_pos, y_pos, graph_width, graph_height) = generate_relative_graph(nNodes, r_trial, size_sec)       
            # Decide which case it is and generate relative node positions
            if (graph_width <= fig_width) & (graph_height <= fig_height): 
                (x_pos, y_pos, graph_width, graph_height) = case1_positions(r_min, tag_list, size_sec, fig_width, fig_height, index_list)
            else: 
                (x_pos, y_pos, graph_width, graph_height) = case2_positions(r_min, tag_list, size_sec, fig_width, fig_height, index_list)
            # Check for overlap by comparing the positions of two neighboring nodes
            overlap = False
            i = 0
            while (overlap == False) and (i < (len(x_pos) - 1)):
                x1, y1 = x_pos[i], y_pos[i]
                x2, y2 = x_pos[i + 1], y_pos[i + 1]
                if np.sqrt((x1 - x2)**2 + (y1 - y2)**2) < size_sec: overlap = True
                i = i + 1
            # Generate absolute node positions in the figure
            xC = abs(min(x_pos)) + size_sec / 2 + (fig_width - graph_width) / 2
            yC = abs(min(y_pos)) + size_sec / 2 + (fig_height - graph_height) / 2
            node_pos_dict = {0: [xC, yC]}
            for n in range(nNodes - 1):
                node_pos_dict[index_list[n + 1]] = [xC + x_pos[n], yC + y_pos[n]]
            node_pos = [[n, int(round(node_pos_dict[n][0])), int(round(node_pos_dict[n][1]))] for n in node_pos_dict.keys()]                           
        # If there is overlap, remove the least popular node, and do it again
        if overlap == True: 
            del tag_list[-1]           
            del index_list[-1]
    return node_pos

# Test

In [13]:
fig_width = 1000
fig_height = 800
size_prim = 200
size_sec = 200
r_min = 200
#tag_list = [0, 12, 1, 4, 2, 1, 2, 1] # Case 2
#tag_list = [0, 1, 2, 3, 3, 4, 1, 1] # Case 1
tag_list = [0, 9, 8, 9, 2, 4, 7, 2, 4, 3, 2, 5, 6, 2, 1, 4, 5, 6, 7, 8, 9, 4, 5, 2, 6, 3, 6, 3, 7, 5, 6, 2, 6, 2]
#tag_list = [0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 2]
#tag_list = [0, 1, 5, 7, 1, 5, 7, 5, 7, 1, 5, 7]

In [14]:
node_positions(tag_list, fig_width, fig_height, size_prim, size_sec, r_min)

[[0, 502, 437],
 [1, 442, 698],
 [2, 230, 306],
 [3, 669, 647],
 [18, 716, 168],
 [19, 230, 568],
 [20, 770, 437],
 [28, 425, 102]]